In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%run init.py

In [33]:
from MLutils import neg_to_bin_labels

In [2]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression

# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
# Rebuild text
from sklearn_bridge import recreate_text_representation
# take text + labels from train ds
from sklearn_bridge import candidate_dict_to_df

In [4]:
from MLutils import report_to_df, train_evaluate

In [5]:
import sklearn, pickle, time, seaborn
import pandas as pd

In [6]:
with open(path_candidate_dict_pickle, 'rb') as f:
    candidate_dict = pickle.load(f)

for splt in candidate_dict.keys():
    print "Split = %i : imported %i candidates" %(splt, len(candidate_dict[splt].keys()))

Split = 0 : imported 12987 candidates
Split = 1 : imported 3230 candidates
Split = 2 : imported 8335 candidates
Split = 3 : imported 79400 candidates


# Load train, dev, test datasets

In [7]:
trim_text = True
trim_window = 5
use_lemmas = True

df_train = candidate_dict_to_df(candidate_dict[0],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )
df_val = candidate_dict_to_df(candidate_dict[1],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )
df_test = candidate_dict_to_df(candidate_dict[2],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )

df_unlab = candidate_dict_to_df(candidate_dict[3],trim_text=trim_text, window=trim_window, lemmas = use_lemmas )


In [8]:
def remove_line_breaks(df):
    df['text'] = map(lambda x: x.replace('\n', ' ' ),df.text)
    return df

In [9]:
df_train = remove_line_breaks(df_train)
train_ids = list(df_train.index)
df_val = remove_line_breaks(df_val)
val_ids = list(df_val.index)
df_test = remove_line_breaks(df_test)
test_ids = list(df_test.index)
df_unlab = remove_line_breaks(df_unlab)
unlab_ids = list(df_unlab.index)

In [10]:
# Export to FastText format

In [11]:
df_train['label1'] = map(lambda label: '__label__%i'%(label), df_train.label)
df_val['label1'] = map(lambda label: '__label__%i'%(label), df_val.label)
df_test['label1'] = map(lambda label: '__label__%i'%(label), df_test.label)
# df_unlab['label1'] = map(lambda label: '__label__%i'%(label), df_unlab.label)


In [12]:
df_train.to_csv('fastText/df_train.txt',header=False,index= False,encoding='utf-8', columns=[ 'label1', 'text'],
               sep= ' ')
df_val.to_csv('fastText/df_val.txt',header=False,index= False,encoding='utf-8', columns=[ 'label1', 'text'],
               sep= ' ')
df_test.to_csv('fastText/df_test.txt',header=False,index= False,encoding='utf-8', columns=[ 'label1', 'text'],
               sep= ' ')
df_unlab.to_csv('fastText/df_unlab.txt',header=False,index= False,encoding='utf-8', columns=[ 'label1', 'text'],
               sep= ' ')


/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/pandas/io/formats/format.py:1586: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  self.obj = self.obj.loc[:, cols]
/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# Run fasttext in cli

In [17]:
# training
! cd fastText/ && ./fasttext supervised -input df_train.txt -output model

Read 0M words
Number of words:  6976
Number of labels: 2
Progress: 100.0% words/sec/thread: 1264066 lr:  0.000000 loss:  0.368285 ETA:   0h 0m


In [18]:
# prediction

In [19]:
! cd fastText/ && ./fasttext predict model.bin df_val.txt > df_val_pred.txt

In [20]:
! cd fastText/ && ./fasttext predict model.bin df_test.txt > df_test_pred.txt

In [21]:
! cd fastText/ && ./fasttext predict model.bin df_unlab.txt > df_unlab_pred.txt

# Read back results

In [22]:
val_preds = pd.read_table('fastText/df_val_pred.txt',header=None)
val_preds.index = df_val.index

test_preds = pd.read_table('fastText/df_test_pred.txt',header=None)
test_preds.index = df_test.index

unlab_preds = pd.read_table('fastText/df_unlab_pred.txt',header=None)
unlab_preds.index = df_unlab.index

In [26]:
#get gold labels (for classif report)
val_gold_labels = map(lambda cid: candidate_dict[1][cid]['label'] , candidate_dict[1].keys())
test_gold_labels = map(lambda cid: candidate_dict[2][cid]['label'] , candidate_dict[2].keys())

# val_pred_labels = map(lambda cid: val_preds_dict[cid] , candidate_dict[1].keys())


In [27]:
len(val_gold_labels)

3230

In [28]:
from sklearn.metrics import classification_report
from MLutils import report_to_df

In [29]:
# print classification_report(val_gold_labels,val_pred_labels)

NameError: name 'val_pred_labels' is not defined

In [40]:
results_dict = {}

# label_*_binary -> list of -1, 1
# label_*_prob_dict -> {40197: 0.96, 40198: 0.03, ... }

#get gold labels (for classif report)
y_val = map(lambda cid: candidate_dict[1][cid]['label'] , candidate_dict[1].keys())
y_test = map(lambda cid: candidate_dict[2][cid]['label'] , candidate_dict[2].keys())

# pass predictions as dict & binary list
y_val_pred_neg = map(lambda x: -1 if x[-2:]=='-1' else 1,val_preds[0])
y_val_pred_dict = dict(zip(df_val.index, neg_to_bin_labels(y_val_pred_bin) ))


y_test_pred_neg = map(lambda x: -1 if x[-2:]=='-1' else 1,test_preds[0])
y_test_pred_dict = dict(zip(df_test.index, neg_to_bin_labels(y_test_pred_neg) ))


y_unlab_pred_neg = map(lambda x: -1 if x[-2:]=='-1' else 1,unlab_preds[0])
y_unlab_pred_dict = dict(zip(df_unlab.index, neg_to_bin_labels(y_unlab_pred_bin) ))


# # y_val_pred_prob = dict(zip(df_val.index,map(lambda x: 1 if x[-2:]=='-1' else 0,val_preds[0]))) #setting primary class as -1

# y_test_pred_neg = map(lambda x: -1 if x[-2:]=='-1' else 1,test_preds[0])
# y_test_pred_bin = dict(zip(df_test.index,map(lambda x: 1 if x[-2:]=='-1' else 0,test_preds[0])))

# y_unlab_pred_bin = map(lambda x: -1 if x[-2:]=='-1' else 1,unlab_preds[0])
# y_unlab_pred_prob = dict(zip(df_unlab.index,map(lambda x: 1 if x[-2:]=='-1' else 0,unlab_preds[0])))


class_report = report_to_df(classification_report(y_val, y_val_pred_neg))
print class_report


# results_dict['fasttext'] = {"label_val_binary" : y_val_pred_bin,
#                             "label_val_prob_dict" : y_val_pred_prob,
#                             "label_test_binary" : y_test_pred_bin,
# #                             "label_test_prob_dict" : y_test_pred_prob,
#                             "label_unlab_binary" : y_unlab_pred_bin,
# #                             "label_unlab_prob_dict" : y_unlab_pred_prob,
#                             #only possible when true labels in place:
# #                                     "confusion_matrix" : conf_matrix,
#                             "classification_report": class_report,
# #                                 "precision": class_report.loc['avg/total','precision'],
# #                                 "recall": class_report.loc['avg/total','recall'],
#                             "f1+": class_report.loc['1','f1-score'],
#                             "model_classes": [-1,1]

#                            }


results_dict['fasttext'] = {"label_val_prob+" : y_val_pred_dict,
                       "label_test_prob+" : y_test_pred_dict,
                       "label_unlab_prob+" : y_unlab_pred_dict,
                        "classification_report": class_report,
                        "f1+": class_report.loc['1','f1-score']
                           }


           precision  recall  f1-score  support
Classes                                        
-1              0.86    0.91      0.88     2525
1               0.59    0.46      0.52      705
avg/total       0.80    0.81      0.80     3230


# Export results_dict

In [41]:
import pickle

In [42]:
with open(path_base_learners+'/fasttext.pkl', 'wb') as f:
    pickle.dump(results_dict, f)